<a href="https://colab.research.google.com/github/GeneralHassaan/Projects/blob/main/PCA_Model_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

HASSAAN ASIF
09/29/2022


In [13]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression

In [2]:
# load the dataset
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784')


In [3]:
from keras.datasets import mnist
# load data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# reshape data
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

11501568/11490434 [==============================] - 0s 0us/step


In [4]:
# scaling data

scaler = StandardScaler()

scaled_df = scaler.fit_transform(X_train)



In [5]:
# pca transformed data
pca = PCA(n_components=784)
pcs = pca.fit_transform(scaled_df)

In [6]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 1,
 'p': 2,
 'weights': 'uniform'}

In [7]:
knn_pipe = make_pipeline(scaler, knn)
knn_pipe.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('kneighborsclassifier', KNeighborsClassifier(n_neighbors=1))])

In [8]:
# trained on pca data
predictions_1 = knn_pipe.predict(pcs)
%timeit predictions_1
predictions_1

24.2 ns ± 1.41 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


array([1, 1, 1, ..., 1, 1, 1], dtype=uint8)

In [9]:
# trained on non-pca data
predictions = knn_pipe.predict(scaled_df)
%timeit predictions
predictions

22.9 ns ± 0.266 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


the non-pca model was the fastest, performing the better in this situation. It delivered about 2 ns faster then the pca data set.

In [11]:
transformer = make_pipeline(StandardScaler(), PCA(n_components=7))

In [15]:
logreg = LogisticRegression()
# Create a modeling pipeline
logreg_pipe = make_pipeline(transformer, logreg)
logreg_pipe.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('standardscaler', StandardScaler()),
                                 ('pca', PCA(n_components=7))])),
                ('logisticregression', LogisticRegression())])

In [16]:
print('Training accuracy:', logreg_pipe.score(X_train, y_train))
print('Testing accuracy:', logreg_pipe.score(X_test, y_test))

Training accuracy: 0.7421833333333333
Testing accuracy: 0.7465
